# Assignment 4

## Constraint satisfaction problems


*Joshua Burris*

Consider the problem of placing $k$ black and $k$ white knights on an
$n \times n$ chess board such that no two knights of the **same** color are attacking each other.  We'll call it the **k-knights problem**.
Your task is to solve this problem by considering it as a constraint satisfaction problem.

**Task 1:** Formulate the problem as a constraint satisfaction problem
with binary constraints.
Describe the variables, their domains, and the constraints.


**Task 2:**
Implement a class called `knights` that is a sub-class of `csp.CSP` (this refers to the class `CSP` within the module `csp.py` in the code library provided with the textbook).  Your implementation of the class needs to allow you to solve the problem with any of the solvers implemented in the `csp` module.


**Task 3:**
Solve the k-knights problem using backtracking search with
the various heuristics we have studied in class and compare their
effectiveness quantified by the number of variable assignments required to get to a solution.

In your experiments, use values of $k$ and $n$ that provide these
methods a challenge and takes sufficient time so that you can obtain some resolution for the comparison of the different heuristics (i.e. $n$ sufficiently large, and $k$ chosen such that it's not too easy to find a solution).

**Task 4:**  The AC3 algorithm. In some cases this algorithm is able to solve constraint satisfaction problems, or at least make some progress towards the solution. Is this the case here?  If not, explain why, and if yes, does it lead to solutions that require a smaller number of variable assignments when used in conjunction with backtracking search?


## Answers

### Task 1: *Formulation of the k-knights problem*

<u>Variables</u>: (B<sub>1</sub>, B<sub>2</sub>,...,B<sub>k</sub>) will be the black and (W<sub>1</sub>, W<sub>2</sub>,...,W<sub>k</sub>) will be the white knights on the chess board. It will be kept in memory as a list of strings. For example: ['B1', 'B2',...,'Bk', 'W1', 'W2',...,'Wk'].

<u>Domains</u>: (D<sub>1</sub>, D<sub>2</sub>, ...,D<sub>k\*2</sub>) is defined by the possible values for the list of variables. On an *n* x *n* chess board these will be the possible tiles that each knight can occupy. To start, each knight has the ability to stand on any tile. We will also keep this information as a dictionary in memory, so for every variable we will assign a list from 0 to n-1. For example: {'W1': [0,2,...,n-1], 'W2': [0,2,...,n-1],..., 'Wk': [0,2,...,n-1], 'B1': [0,2,...,n-1], 'B2': [0,2,...,n-1],..., 'Bk': [0,2,...,n-1]}.

<u>Constraints</u>: (C<sub>1</sub>, C<sub>2</sub>,...,C<sub>m</sub>) will be defined such that no two knights of the **same** color are able to attack each other, and that no two knights occupy the same tile space. For example: Tile of B<sub>1</sub> ≠ Tile of B<sub>3</sub>.


### Task 2: *Implementation*


In [28]:
# complete the implementation of the following class so it is able
# to solve the k-knights problem.

import csp
import time

class knights(csp.CSP) :
    def __init__(self, k, n) :
        self.k = k
        self.n = n
        
        self.moves = {'UP': -n, 'DOWN': n, 'RIGHT': 1, 'LEFT': -1}
        self.possible_moves = [self.moves['UP']*2 + self.moves['LEFT'],
             self.moves['UP']*2 + self.moves['RIGHT'],
             self.moves['LEFT']*2 + self.moves['UP'],
             self.moves['LEFT']*2 + self.moves['DOWN'],
             self.moves['RIGHT']*2 + self.moves['UP'],
             self.moves['RIGHT']*2 + self.moves['DOWN'],
             self.moves['DOWN']*2 + self.moves['LEFT'],
             self.moves['DOWN']*2 + self.moves['RIGHT']]
        
        self.variables = []
        #for i in range(k) :
        #    self.variables += ['W'+str(i+1), 'B'+str(i+1)]
        for i in range(k) :
            self.variables += ['W'+str(i+1)]
        for i in range(k) :
            self.variables += ['B'+str(i+1)]
        
        self.domains = {}
        for var in self.variables :
            self.domains[var] = [i for i in range(n*n)]
        
        self.neighbors = {}
        for v in self.variables :
            l = []
            for j in self.variables :
                if v != j :
                    l.append(j)
            self.neighbors[v] = l
        
        def constr(A, a, B, b) :
            if a == b :
                return False
            if (A[0] == 'W' and B[0] == 'W') or (A[0] == 'B' and B[0] == 'B') :
                for i in self.possible_moves :
                    if a+i == b :
                        return False
            return True
        self.constraints = constr
        
        self.initial = ()
        self.curr_domains = None
        self.nassigns = 0

    def display(self, assignment) :
        board = ['∙']*(self.n*self.n)
        for key, value in assignment.items() :
            board[value] = key[0]
        print('―'*(self.n*2+1))
        for i in range(self.n) :
            print('|', end='')
            for j in range(self.n) :
                print(board[self.n*i + j], end='|')
            print()
            print('―'*(self.n*2+1))

### Task 3: *Experiments on the k-knights problem*

In [29]:
# code for solving the k-knights problem as a CSP problem..

nklist = [(4, 4), (5, 10), (5, 12), (6, 12), (8, 24), (10, 40), (12, 60), (14, 84), (16, 100), (16, 112)]

index = 1
n = nklist[index][0]
k = nklist[index][1]

problem = knights(k, n)
print('Heuristic: None')
start = time.time()
result = csp.backtracking_search(problem)
end = time.time() - start
print('Time (seconds):', end)
print('Assignments:', problem.nassigns)
print('Solution:')
problem.display(result)
print()
del problem

problem = knights(k, n)
print('Heuristic: Most constrained variable')
start = time.time()
result = csp.backtracking_search(problem,
                                select_unassigned_variable=csp.mrv)
end = time.time() - start
print('Time (seconds):', end)
print('Assignments:', problem.nassigns)
print('Solution:')
problem.display(result)
print()

problem = knights(k, n)
print('Heuristic: Least constraining value')
start = time.time()
result = csp.backtracking_search(problem,
                                order_domain_values=csp.lcv)
end = time.time() - start
print('Time (seconds):', end)
print('Assignments:', problem.nassigns)
print('Solution:')
problem.display(result)
print()

problem = knights(k, n)
print('Heuristic: Forward checking')
start = time.time()
result = csp.backtracking_search(problem,
                                inference=csp.forward_checking)
end = time.time() - start
print('Time (seconds):', end)
print('Assignments:', problem.nassigns)
print('Solution:')
problem.display(result)
print()

Heuristic: None
Time (seconds): 0.9300222396850586
Assignments: 2027
Solution:
―――――――――――
|W|W|W|B|∙|
―――――――――――
|B|W|B|∙|B|
―――――――――――
|∙|B|∙|B|W|
―――――――――――
|∙|W|B|W|B|
―――――――――――
|W|B|W|B|W|
―――――――――――

Heuristic: Most constrained variable
Time (seconds): 0.009557247161865234
Assignments: 20
Solution:
―――――――――――
|B|W|B|W|B|
―――――――――――
|W|B|W|B|W|
―――――――――――
|B|W|B|W|B|
―――――――――――
|W|B|W|B|W|
―――――――――――
|∙|∙|∙|∙|∙|
―――――――――――

Heuristic: Least constraining value
Time (seconds): 1.746643304824829
Assignments: 2027
Solution:
―――――――――――
|W|W|W|B|∙|
―――――――――――
|B|W|B|∙|B|
―――――――――――
|∙|B|∙|B|W|
―――――――――――
|∙|W|B|W|B|
―――――――――――
|W|B|W|B|W|
―――――――――――

Heuristic: Forward checking
Time (seconds): 0.07255125045776367
Assignments: 2027
Solution:
―――――――――――
|W|W|W|B|∙|
―――――――――――
|B|W|B|∙|B|
―――――――――――
|∙|B|∙|B|W|
―――――――――――
|∙|W|B|W|B|
―――――――――――
|W|B|W|B|W|
―――――――――――



<u>Results</u>:

<b>*n* = 5, *k* = 10</b>

| Heuristic                 | Assignments | Time (sec) |
|-|-|-|
| None                      | 2027        | 0.93002    |
| Most constrained variable | 20          | 0.00956    |
| Least constraining value  | 2027        | 1.74664    |
| Forward checking          | 2027        | 0.07255    |

<u>Discussion</u>:

I chose to use 5 as the number for *n* and 10 for *k* because I think it takes sufficiently long to run (~1 sec). Anything larger seemed to take too long to solve. For example: *n* = 6 & *k* = 12 took more than 60 seconds for no heuristics to solve. The "most constrained variable" heuristic took varying assignments to solve when I ran it, the least I found was 20 when the others took about 2000, although it generally took way longer than the others.

I purposefully chose to write the variables list in my knights class as all white then all black knights, instead of using alternating colors. Alternating colors is already very close to the solution so the number of assignments will be the same for all heuristics. It could solve *n* = 20 and *k* = 100 in less than 1 second.

Forward checking was the fastest heuristic on average because it can prune branches that we find to be impossible so we don't have to check them.

I'm not including statistics when using all three heuristics together because it takes significantly longer to run than when using any other single one. For example: when *n* = 5 & *k* = 12 it takes 0.00736 seconds to solve with no heuristics, but 21.19605 seconds to solve with all heuristics. And also the number of assignments was over 6000 times more.


### Task 4: *The AC3 algorithm*

No, the AC3 algorithm is not able to solve the knights problem because it cannot prune domains to an extent where it can definitively make any progress or even place a knight on the board. It will terminate when it can't prune anymore domains from the variables, but because it can't prune enough domains to say that any knight must go to any one tile it will terminate before progress is made. This is because all white or black knights can be swapped around for each other of the same color and still be valid.

## Submission

Answer the questions in the cells reserved for that purpose.
Submit your report as a Jupyter notebook via Canvas. Running the notebook should generate all the results in your notebook.  Leave the output of your notebook intact so we don't have to run it to see the results.

## Grading

Grading will be based on the following criteria:

* Your code solves the specified problem.
* Overall readability and organization of the notebook.
* Effort in making interesting observations where required.
* Conciseness. Points may be taken off if the notebook is overly long.